# The geometry of somatosensory representations in the cortex

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%matplotlib widget

In [ ]:
from draw_data_map import cont_body_cmap, disc_cmap, cont_cmap
from gradient_utils import draw_surface, add_map, rotated_hsv, pickle_load, get_parc_vertices
import numpy as np
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
from loadmat import save_mat, save_fig, load_mat, set_folder
import shutil

set_folder('./MaxR/')
NUM_OF_VERTICES = 163842

areas = {'s1' : ['3a', '3b', '1', '2'],
         'v1': ['V1'],
         'a1' : ['A1'],
         '4': ['4'],
         '6': ['6mp','6ma','6a','6d','6r','6v'],
         '7': ['7Am','7PL','7Pm','7AL','7PC','7m'],
        'parietal' : ['5L', '7AL', '7Am', '7PL', '7PC', 'VIP', 'PFt', 'AIP', 'PFop', 'PF'],
        'm1' : ['4', '23d'],
        'frontal' : ['FEF', 'PEF', '55b', '6ma', '6d', '6mp', '6v', '6r', '6a'],
        'medial' : ['5m', '5mv', '23c', '24dd', '24dv', 'SCEF', 'p24pr', 'a24pr', 'p32pr'],
        'insula' : ['52','PI','Ig','PoI1','PoI2','FOP2','FOP3','MI','AVI','AAIC','Pir','FOP4','FOP5']
        }
s1_areas_num = [9,51,52,53]
insula_areas_num = [103, 106, 108, 109, 111, 112, 114, 115, 167, 168, 169, 178]

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# style = """
#     <style>
#        .jupyter-widgets-output-area .output_scroll {
#             height: unset !important;
#             border-radius: unset !important;
#             -webkit-box-shadow: unset !important;
#             box-shadow: unset !important;
#         }
#         .jupyter-widgets-output-area  {
#             height: auto !important;
#         }
#     </style>
#     """
# display(widgets.HTML(style))

hemi_dd = widgets.Dropdown(options=[('left','lh'),('right', 'rh')], value='lh', description='Hemi:')
lim_dd = widgets.Dropdown(options=[90, 120, 150], value=90, description='Limit Angles:')
type_dd =  widgets.Dropdown(options=[('Selectivity','sel'), ('Laterality','lat')], value='sel', description='Hierarchy Type:')
map_dd =  widgets.Dropdown(options=[('Selectivity','sel'), ('Laterality','lat'), ('Body','body')], value='sel', description='Map:')
body_dd = widgets.Dropdown(options=[('Sel-Lat',0), ('Body',1)], value=0, description='Map Type:')
area_dd = widgets.Dropdown(options=[('All','All')]+[(key,key) for key in areas.keys()], value='All', description='Area:')
surf_dd = widgets.Dropdown(options=[('2-D', 'flat'), ('3-D', 'multi')], value='multi', description='Surface')
mask_dd = widgets.Dropdown(options=[('MaxR', 'maxR'), ('Random', 'RandomEffect')], value='maxR', description='Mask')
body_smooth_dd = widgets.Dropdown(options=[('0', ''), ('8','8'),('100', '100'), ('200', '200'), ('400', '400')], value='200', description='Body Smooth')
sel_smooth_dd = widgets.Dropdown(options=[('800', '800'), ('1000', '1000')], value='800', description='Sel Smooth')
lat_smooth_dd = widgets.Dropdown(options=[('400', '400'), ('800', '800'), ('1000', '1000')], value='400', description='Lat Smooth')
multarea_ms = widgets.SelectMultiple(options=[(key,key) for key in areas.keys()], value=['s1'], description='Stacked Areas:')
clear_cb = widgets.Checkbox(value=False, description='Clear Output')
drawMap_cb = widgets.Checkbox(value=True, description='Draw Map')
multHist_cb = widgets.Checkbox(value=False, description='4 areas hist')
hist_cb = widgets.Checkbox(value=False, description='hist')
exS1_cb = widgets.Checkbox(value=False, description='S1')
exS1a_cb = widgets.Checkbox(value=False, description='Exclude S1')
exIP_cb = widgets.Checkbox(value=False, description='IP')
exIns_cb = widgets.Checkbox(value=False, description='Insula')
count_cb = widgets.Checkbox(value=True, description='Count')
whitebg_cb = widgets.Checkbox(value=False, description='White bg')
smooth_cb = widgets.Checkbox(value=True, description='Smooth Map')
areaLines_cb = widgets.Checkbox(value=False, description='Areas Lines')
s1Line_cb = widgets.Checkbox(value=False, description='S1 Line')
insulaLine_cb = widgets.Checkbox(value=False, description='Insula Line')
title_cb = widgets.Checkbox(value=False, description='Title')
contour_cb = widgets.Checkbox(value=False, description='Contour')
cbar_cb = widgets.Checkbox(value=False, description='Color Bar')

run_bt = widgets.Button(description='Run')

In [ ]:
import matplotlib.ticker

class OOMFormatter(matplotlib.ticker.ScalarFormatter):
    def __init__(self, order=0, fformat="%1.1f", offset=True, mathText=True):
        self.oom = order
        self.fformat = fformat
        matplotlib.ticker.ScalarFormatter.__init__(self,useOffset=offset,useMathText=mathText)
    def _set_order_of_magnitude(self):
        self.orderOfMagnitude = self.oom
    def _set_format(self, vmin=None, vmax=None):
        self.format = self.fformat
        if self._useMathText:
            self.format = r'$\mathdefault{%s}$' % self.format

def get_mean_var_std(theta):
    rad_t = np.deg2rad(theta)
    mean_sin = np.mean(np.sin(rad_t))
    mean_cos = np.mean(np.cos(rad_t))
    mean = np.rad2deg(np.arctan2(mean_sin,mean_cos))
    R = np.sqrt((mean_sin**2)+(mean_cos**2))
    var = 1 - R
    std = np.sqrt(-2*np.log(R))
    return np.around(mean,5), np.around(var,5), np.around(std,5)

def circular_pdf(mu, var):
    mu = np.deg2rad(mu)
    x = np.linspace(-np.pi,np.pi, 361)
    y = np.exp(np.cos(x-mu)/var)/(iv(0,1/var)*2*np.pi)
    return y

# Main Maps

In [ ]:
def main_maps(b):
    set_folder(f'./{mask_dd.value}/')
    hemi = hemi_dd.value
    map_type = type_dd.value
    borders = np.zeros(NUM_OF_VERTICES)
    with out_main_maps:
        if map_dd.value=='body':
            if smooth_cb.value:
                if body_smooth_dd.value:
                    under_score = '_'
                else:
                    under_score = ''
                title = f'body_{hemi}{under_score}{body_smooth_dd.value}'
                cmap = cont_body_cmap
                data = load_mat(f'MapsCalculation/body_{hemi}{under_score}{body_smooth_dd.value}')
                cont_file_name = f'body_contour_{hemi}{under_score}{body_smooth_dd.value}.mat'
            else:
                title = f'body_disc_{hemi}'
                cmap = disc_cmap
                data = load_mat(f'MapsCalculation/body_disc_{hemi}')
        else:
            cmap = cont_cmap
            if smooth_cb.value:
                title = f'{map_dd.value}_{hemi}_{eval(f"{map_dd.value}_smooth_dd.value")}'
                data = load_mat(f'MapsCalculation/{map_dd.value}_{hemi}_{eval(f"{map_dd.value}_smooth_dd.value")}')
                cont_file_name = f'{map_dd.value}_contour_{hemi}_{eval(f"{map_dd.value}_smooth_dd.value")}.mat'
            else:
                title = f'{map_dd.value}_{hemi}'
                data = load_mat(f'MapsCalculation/{map_dd.value}_{hemi}')
            
        if contour_cb.value:
            borders = load_mat(cont_file_name)
            # dataNZ = data[np.nonzero(data)]
            # dataNZind = np.argwhere(data).squeeze()
            # for j in range(16):
            #     lower = np.quantile(dataNZ,0.0625*j+0.025)
            #     upper = np.quantile(dataNZ,0.0625*j+0.0375)
            #     tempind = np.argwhere((data[dataNZind] > lower) & (data[dataNZind] < upper)).squeeze()
            #     a = dataNZind[tempind].sum()
            #     borders[dataNZind[tempind]] = 255
            # save_mat(cont_file_name, {'data': borders})
        
        if area_dd.value=='All':
            sigInd = load_mat(f'MapsCalculation/sigInd_{hemi}.mat').astype(bool)
        else:
            sigInd = np.zeros(NUM_OF_VERTICES).astype(bool)
            sigInd[get_parc_vertices(hemi, areas[area_dd.value])] = True
        data[~sigInd] = 0
        if data.min() < 0:
            threshold = data.min() - 1
            data[data == 0] = threshold - 0.1
        else:
            threshold = 0.00001
            
        if whitebg_cb.value:
            bg_map = np.zeros_like(data)
            shutil.copyfile('./surface-plot-utils.white_bg.js',
                            './surface-plot-utils.js')
        else:
            bg_map = None
        
        if areaLines_cb.value or s1Line_cb.value:
            bordersList = pickle_load(f'./data/parcellation_map_borders_{hemi}.pkl', use_main_folder=False)
        if areaLines_cb.value:
            for region in bordersList.keys():
                border = np.array(list(bordersList.get(region)))
                borders[border] = 255
        elif s1Line_cb.value:
                for region in bordersList.keys():
                    two_areas_num = region.split('-')
                    #if (int(two_areas_num[0]) not in s1_areas_num or int(two_areas_num[1]) not in s1_areas_num) and\
                    #(int(two_areas_num[0]) in s1_areas_num or int(two_areas_num[1]) in s1_areas_num):
                    if (int(two_areas_num[0]) in s1_areas_num or int(two_areas_num[1]) in s1_areas_num):
                        border = np.array(list(bordersList.get(region)))
                        borders[border] = 255
    
        title = title if title_cb.value else ''
        return_info = (areaLines_cb.value or contour_cb.value or s1Line_cb.value)
        info = draw_surface(data, hemi=hemi, surf=surf_dd.value, title=title, cmap=cmap, threshold=threshold,
                            return_info = return_info, colorbar=cbar_cb.value, bg_map = bg_map)
        if return_info:
            add_map(info, borders, hemi, threshold=-1.1, cmap=ListedColormap(['black', "#000001"]))
        if whitebg_cb.value:
            shutil.copyfile('./surface-plot-utils.js',
                            './nilearn/plotting/data/js/surface-plot-utils.js')


out_main_maps = widgets.Output()
run_bt._click_handlers.callbacks = []
run_bt.on_click(main_maps)
display(widgets.VBox([widgets.HBox([widgets.VBox([hemi_dd, map_dd, area_dd, surf_dd]), 
                                    widgets.VBox([sel_smooth_dd, lat_smooth_dd, body_smooth_dd]), mask_dd]),
                     widgets.HBox([smooth_cb, contour_cb, whitebg_cb]),
                     widgets.HBox([areaLines_cb, s1Line_cb]),
                     widgets.HBox([title_cb, cbar_cb]) , run_bt, out_main_maps]))


# Angle maps

In [ ]:
def angle_maps(b):
    set_folder(f'./{mask_dd.value}/')
    with out_area_maps:
        if clear_cb.value:
            plt.close('all')
            clear_output()
        hemi = hemi_dd.value
        if not body_dd.value:
            data = load_mat(f'sel{sel_smooth_dd.value}-lat{lat_smooth_dd.value}_{hemi}.mat')
            map_name = f'sel{sel_smooth_dd.value}-lat{lat_smooth_dd.value}_{area_dd.value}_{hemi}'
        else:
            data = load_mat(f'{type_dd.value}{eval(f"{type_dd.value}_smooth_dd.value")}-body{body_smooth_dd.value}_{hemi}.mat')
            map_name = f'{type_dd.value}{eval(f"{type_dd.value}_smooth_dd.value")}-body{body_smooth_dd.value}_{area_dd.value}_{hemi}'
    
        sigInd = load_mat(f'MapsCalculation/sigInd_{hemi}.mat').astype(bool)
        if area_dd.value!='All':
            area = np.zeros(NUM_OF_VERTICES).astype(bool)
            area[get_parc_vertices(hemi, areas[area_dd.value])] = True
            sigInd = sigInd & area
        
        data[~sigInd] = -181
        #data[(data>80)&(data<100)] = -181
        
        # if area_dd.value=='all':
        #     if exS1a_cb.value:
        #         sigInd = np.setdiff1d(sigInd,get_parc_vertices(hemi, areas['s1']))
        # else:
        #     sigInd = np.intersect1d(sigInd, get_parc_vertices(hemi, areas[area_dd.value]))
        if whitebg_cb.value:
            bg_map = np.zeros_like(data)
            shutil.copyfile('./surface-plot-utils.white_bg.js',
                            './surface-plot-utils.js')
        else:
            bg_map = None 
        
        title = map_name if title_cb.value else ''
        return_info = (areaLines_cb.value or s1Line_cb.value)
        if drawMap_cb.value:
            info = draw_surface(data, hemi=hemi, title=title, cmap=rotated_hsv, threshold=-180.5,
                                return_info = return_info, surf = surf_dd.value, 
                                colorbar=cbar_cb.value, bg_map = bg_map, vmin = -180, vmax = 180)
            
            if areaLines_cb.value or s1Line_cb.value:
                bordersList = pickle_load(f'./data/parcellation_map_borders_{hemi}.pkl', use_main_folder=False)
                borders = np.zeros(NUM_OF_VERTICES)
                if area_dd.value!='s1' and not s1Line_cb.value:
                    for region in bordersList.keys():
                        border = np.array(list(bordersList.get(region)))
                        borders[border] = 1
                else:
                    for region in bordersList.keys():
                        two_areas_num = region.split('-')
                        if (int(two_areas_num[0]) in s1_areas_num or int(two_areas_num[1]) in s1_areas_num):
                            border = np.array(list(bordersList.get(region)))
                            borders[border] = 1
    
                add_map(info, borders, cmap='black_pink', hemi=hemi)

        if hist_cb.value:
            plotRange=np.arange(-180, 181, 90)
            fig, ax = plt.subplots(figsize=(12, 12), layout='tight')
            g = sns.histplot(data[sigInd], kde=False, stat='count', bins=90, ax=ax, color='black', label=area_dd.value)
            print(f'{map_name}')
            m, v, s = get_mean_var_std(data[sigInd])
            print(f'{area_dd.value}\n\u03BC: {m}\nVAR: {v}\n\u03C3: {s}\n')
            # df = pd.concat([pd.DataFrame(angles[si], columns=[sel_areas[i]]) for i,si in enumerate(sigIndAreas)], axis=1)
            if exS1a_cb.value:
                sigInd = np.intersect1d(np.where(sigInd)[0],get_parc_vertices(hemi, areas['s1']))
    
                sns.histplot(data[sigInd], kde=False, stat='count', bins=90, ax=ax, facecolor='#FFF', label='S1')
                m, v, s = get_mean_var_std(data[sigInd])
                print(f'S1\n\u03BC: {m}\nVAR: {v}\n\u03C3: {s}\n')
            #ax.set_xlabel("Angle", fontdict={'size': 72, 'family':"Helvetica"})
            ax.set_ylabel("")
            ax.set_ylabel("Count", fontdict={'size': 72, 'family':"Helvetica"})
            ax.set_title("   Body-Laterality   ", fontdict={'size': 72, 'family':"Helvetica"})
            #ax.set_yticks(np.arange(0,1100,200))
            ax.set_xticks(plotRange)
            #ax.yaxis.set_major_formatter(OOMFormatter(2, "%1.0f"))
            ax.ticklabel_format(axis='y', scilimits=[-5,2], useMathText=True)
            ax.yaxis.offsetText.set_fontsize(36)
            ax.tick_params(labelsize=36)
            if area_dd.value=='All':
                plt.legend()
                plt.setp(g.get_legend().get_texts(), fontsize='32') 
            #save_fig(plt, f'{map_name}.png', bbox_inches='tight')
            plt.show()
        if whitebg_cb.value:
            shutil.copyfile('./surface-plot-utils.js',
                            './surface-plot-utils.js')

out_area_maps = widgets.Output()
run_bt._click_handlers.callbacks = []
run_bt.on_click(angle_maps)
display(widgets.VBox([widgets.HBox([widgets.VBox([hemi_dd, type_dd, body_dd, area_dd, surf_dd]),
                                    widgets.VBox([sel_smooth_dd, lat_smooth_dd, body_smooth_dd]), mask_dd]),
                      widgets.HBox([clear_cb, drawMap_cb, whitebg_cb]),
                      widgets.HBox([areaLines_cb, s1Line_cb, cbar_cb]),
                      widgets.HBox([hist_cb, exS1a_cb, title_cb]), run_bt,out_area_maps]))


# mean std for all maps

In [ ]:
set_folder('./MaxR/')  # RandomEffect or MaxR
for hemi in ['lh', 'rh']:
    for map_name in [f'sel-lat_{hemi}', f'sel-body_{hemi}', f'lat-body_{hemi}']:
        for area in ['All', 'S1']:
            data = load_mat(f'{map_name}.mat')
            
            sigInd = np.where(load_mat(f'MapsCalculation/sigInd_{hemi}.mat').astype(bool))[0]
            if area!='All':
                sigInd = np.intersect1d(sigInd, get_parc_vertices(hemi, areas[area]))
           
            print(f'{map_name} {area}')
            m, v, s = get_mean_var_std(data[sigInd])
            print(f'\u03BC: {m}\nVAR: {v}\n\u03C3: {s}\n')
